# Image classification using EEG Data 

#### Import Libraries 

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import OneHotEncoder
import numpy as np 
import tensorflow as tf 
import keras 
import random 

#### Prepare Data

In [2]:
spectrogram_data = np.load('data/spectrogram.npy')
labels = pd.read_csv('data/labels_desc.csv')

labelEncoder = OneHotEncoder()
labelEncoder.fit(labels['label'].to_numpy().reshape(-1, 1))

train_df = labels[labels['split'] == 'Train']
test_df = labels[labels['split'] == 'Test']

train_df_idxs = train_df.index.to_list()
test_df_idxs = test_df.index.to_list()

x_train, y_train = spectrogram_data[train_df_idxs], labelEncoder.transform(train_df['label'].to_numpy().reshape(-1, 1)).toarray()
x_test, y_test = spectrogram_data[test_df_idxs], labelEncoder.transform(test_df['label'].to_numpy().reshape(-1, 1)).toarray()

#### Model Definition

In [ ]:
inputs = keras.Input((14, 33, 62))

x = keras.layers.Conv2D(92, 5, 2, activation='leaky_relu')(inputs)  # <-- Conv2D
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Conv2D(128, 5, 2, activation='leaky_relu')(x)      # <-- Conv2D
x = keras.layers.Dropout(0.2)(x)

x = keras.layers.Flatten()(x)                                       # <-- Flatten

x = keras.layers.Dense(256, activation='leaky_relu')(x)             # <-- Dense
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Dense(128, activation='leaky_relu')(x)             # <-- Dense
x = keras.layers.Dense(64, activation='leaky_relu')(x)              # <-- Dense

outputs = keras.layers.Dense(40, activation='softmax')(x)           # <-- Dense

# Create Model
model = keras.Model(inputs, outputs)

# Visualize Model
print(model.summary())
keras.utils.plot_model(model, show_shapes=True)

#### Compile and Train Model

In [ ]:
model.compile(keras.optimizers.Adam(1e-3), 
              keras.losses.CategoricalCrossentropy(), 
              metrics=['categorical_accuracy'])

model.fit(x_train, y_train, 
          batch_size=48, epochs=100, 
          validation_data=(x_test, y_test))

#### Evaluate Model

In [ ]:
loaded_model = keras.models.load_model('models/savedModel.keras')
test_loss, test_accuracy = loaded_model.evaluate(x_test, y_test)

print(f"\n\nMulti-class Accuracy on Test-Set: {round(test_accuracy*100, 2)}%")